In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        # self.fc1 = nn.Linear ( ??? , 512)
        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    def forward (self,x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)

In [7]:
import os
MODEL_NAME = "model-1590211477"
PATH = os.path.join(os.getcwd(), MODEL_NAME)
model = Net()
model.load_state_dict(torch.load(PATH))
model.eval()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [8]:
training_data = np.load("training_data.npy", allow_pickle=True)
print("Train data length :",len(training_data))

Train data length : 24946


In [14]:
#mocking data coming into the system
#input data is in np array
index = 25
input_X, labelled_y = training_data[index] 
# convert numpy to tensor - note single tensor should have the dimention [1,50,50]
# Note: input_X, labeled_y = test_X[index:index+1],test_y[index:index+1]
input_X = torch.Tensor([input_X]).view(-1,50,50)
labelled_y = torch.Tensor([labelled_y])
# Normalisation value from 0 - 1
input_X = input_X / 255.0
with torch.no_grad():
    outputs = model(input_X.view(-1,1,50,50))

match = [torch.argmax(i) == torch.argmax(j) for i,j in zip(outputs,labelled_y)]
print(match)
print("input X in tensor value ", input_X)
print("labelled y in tensor ", labelled_y)
print("output from the model ", outputs)


[tensor(True)]
input X in tensor value  tensor([[[0.7020, 0.6627, 0.7294,  ..., 0.4000, 0.6157, 0.4078],
         [0.5294, 0.6431, 0.8314,  ..., 0.5843, 0.5098, 0.4431],
         [0.5608, 0.6902, 0.4902,  ..., 0.4784, 0.4706, 0.4627],
         ...,
         [0.3020, 0.1882, 0.5098,  ..., 0.3529, 0.2745, 0.2941],
         [0.4392, 0.3490, 0.5294,  ..., 0.3569, 0.2706, 0.1569],
         [0.4471, 0.3882, 0.3686,  ..., 0.3686, 0.2157, 0.2784]]])
labelled y in tensor  tensor([[0., 1.]])
output from the model  tensor([[0.1646, 0.8354]])
